In [2]:
import os

import pandas as pd
import numpy as np
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import scipy.stats as scs

from scipy.optimize import minimize
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [32]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
init_notebook_mode(connected = True)

In [33]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [4]:
df = pd.read_csv("../data/labeled_data.csv", sep=',')
df['created_at'] = df['created_at'].apply(pd.Timestamp)

In [5]:
districts = ['871e7689cffffff','871e76883ffffff','871e76882ffffff','871e7689dffffff','871e768b1ffffff',
             '871e76135ffffff', '871e76899ffffff','871e76895ffffff','871e76898ffffff','871e76126ffffff',
             '871e76880ffffff','871e76881ffffff','871e76885ffffff','871e7689affffff','871e7688affffff',
             '871e76134ffffff','871e76c6dffffff','871e76891ffffff','871e7689effffff','871e76893ffffff',
             '871e7689bffffff','871e7688effffff','871e768b3ffffff','871e76890ffffff','871e76c69ffffff',
             '871e768aaffffff','871e76888ffffff','871e76886ffffff','871e7688cffffff','871e768b5ffffff',
             '871e76124ffffff','871e768abffffff','871e76c68ffffff','871e76c6cffffff','871e768a8ffffff',
             '871e76122ffffff','871e76136ffffff','871e76130ffffff','871e76884ffffff','871e768aeffffff',
             '871e76c6bffffff','871e76120ffffff','871e7688bffffff','871e76894ffffff']

In [6]:
class HoltWinters:       
    def __init__(self, series, slen, alpha, beta, gamma, n_preds, scaling_factor=1.96):
        self.series = series
        self.slen = slen
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.n_preds = n_preds
        self.scaling_factor = scaling_factor
        
        
    def initial_trend(self):
        sum = 0.0
        for i in range(self.slen):
            sum += float(self.series[i+self.slen] - self.series[i]) / self.slen
        return sum / self.slen  
    
    def initial_seasonal_components(self):
        seasonals = {}
        season_averages = []
        n_seasons = int(len(self.series)/self.slen)
        # вычисляем сезонные средние
        for j in range(n_seasons):
            season_averages.append(sum(self.series[self.slen*j:self.slen*j+self.slen])/float(self.slen))
        # вычисляем начальные значения
        for i in range(self.slen):
            sum_of_vals_over_avg = 0.0
            for j in range(n_seasons):
                sum_of_vals_over_avg += self.series[self.slen*j+i]-season_averages[j]
            seasonals[i] = sum_of_vals_over_avg/n_seasons
        return seasonals   

          
    def triple_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.Season = []
        self.Trend = []
        self.PredictedDeviation = []
        self.UpperBond = []
        self.LowerBond = []
        
        seasonals = self.initial_seasonal_components()
        
        for i in range(len(self.series)+self.n_preds):
            if i == 0: # инициализируем значения компонент
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i%self.slen])
                
                self.PredictedDeviation.append(0)
                
                self.UpperBond.append(self.result[0] + 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                
                
                self.LowerBond.append(self.result[0] - 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                
                
                
                continue
            if i >= len(self.series): # прогнозируем
                m = i - len(self.series) + 1
                self.result.append((smooth + m*trend) + seasonals[i%self.slen])
                
                # во время прогноза с каждым шагом увеличиваем неопределенность
                self.PredictedDeviation.append(self.PredictedDeviation[-1]*1.01) 
                
            else:
                val = self.series[i]
                last_smooth, smooth = smooth, self.alpha*(val-seasonals[i%self.slen]) + (1-self.alpha)*(smooth+trend)
                trend = self.beta * (smooth-last_smooth) + (1-self.beta)*trend
                seasonals[i%self.slen] = self.gamma*(val-smooth) + (1-self.gamma)*seasonals[i%self.slen]
                self.result.append(smooth+trend+seasonals[i%self.slen])
                
                # Отклонение рассчитывается в соответствии с алгоритмом Брутлага
                self.PredictedDeviation.append(self.gamma * np.abs(self.series[i] - self.result[i]) 
                                               + (1-self.gamma)*self.PredictedDeviation[-1])
                
            
            self.UpperBond.append(self.result[-1] + 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])


            self.LowerBond.append(self.result[-1] - 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])
                

In [10]:
def get_predicted_data(df_district):
  
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []

        values = data.values
        alpha, beta, gamma = x

        # задаём число фолдов для кросс-валидации
        tscv = TimeSeriesSplit(n_splits=3) 

        # идем по фолдам, на каждом обучаем модель, строим прогноз на отложенной выборке и считаем ошибку
        for train, test in tscv.split(values):

            model = HoltWinters(series=values[train], slen = 24*7, alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()

            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)
        return np.mean(np.array(errors)) 
  
    df_district = pd.DataFrame(df_district['ride_id'].groupby(df_district.created_at).count())
    df_district = df_district.resample('H', how='count')
    
    df_full = pd.DataFrame(index = pd.date_range(start='2018-08-21 00:00:00', end='2019-02-21 16:00:00', freq='H'))
    s=[]
    t = df_district.index.tolist()
    right = df_full.index.tolist()
    for time in right:
        if time in t:
            s.append(df_district.loc[time].ride_id)
        else:
            s.append(0)

    df_full['count'] = s

    #Normalization
    df_normilized = np.log(df_full+1)
    data = df_normilized['count'][:-600] # отложим часть данных для тестирования
    x = [0, 0, 0] # инициализируем значения параметров
    opt = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))

    # Из оптимизатора берем оптимальное значение параметров
    alpha_final, beta_final, gamma_final = opt.x
    #print(alpha_final, beta_final, gamma_final)
    
    d = df_normilized['count']
    model = HoltWinters(d, slen = 24*7, alpha = alpha_final, beta = beta_final, gamma = gamma_final, n_preds = 7*24, scaling_factor = 2.56)
    model.triple_exponential_smoothing()
    
    df_with_forecast = pd.DataFrame(index = pd.date_range(start='2018-08-21 00:00:00', end='2019-02-28 16:00:00', freq='H'))
    df_with_forecast['model'] = [np.exp(x)+1 for x in model.result]
    df_with_forecast['actual'] = df_full['count'].tolist() + [0]*(7*24)
    return df_with_forecast, alpha_final, beta_final, gamma_final
    

In [ ]:
maes = {}
coefs = {}
days_back = 7

df_predictions = pd.DataFrame(index = pd.date_range(start='2018-08-21 00:00:00', end='2019-02-28 16:00:00', freq='H'))


for index, district in enumerate(districts):
    print(index)
    df_district = df[df.pickup_district==district]
    df_with_forecast, alpha, beta, gamma = get_predicted_data(df_district)
    
    maes[district] = mean_absolute_error(df_with_forecast[4433-days_back*24:4433]['actual'], df_with_forecast[4433-days_back*24:4433]['model'])
    coefs[district] = (alpha, beta, gamma)
    df_predictions[district + '_model'] = df_with_forecast['model']
    df_predictions[district + '_actual'] = df_with_forecast['actual']

In [ ]:
# коефіцієнти моделі (альфа, бета, гама) для кожного округу
coefs

In [ ]:
# MAE для кожного округу за останні 7 історично відомих днів
maes

In [ ]:
# датасет із вхідними даними і прогнозоми моделі для кожного округу (+ вперед на 7 днів прогноз кожну годину)
df_predictions.head()

In [ ]:
df_predictions.tail()

In [ ]:
df_predictions.to_csv('predictions.csv', sep=',')

In [29]:
def plot(district):
    df_test = df_predictions[[district+'_model', district+'_actual']]
    return df_test

In [30]:
test_1 = plot('871e7689cffffff')

In [37]:
def plot_same_scale(df, var=[], title=''):
    data = []
    for v in var:
        trace = go.Scatter( x = df.index, y = df[v], mode = 'lines', name = v)
        data.append(trace)
    
    layout = go.Layout(title=title, 
              xaxis=dict(title=title, titlefont=dict(family='Courier New, monospace',size=14)))
              
    fig = go.Figure(data = data, layout=layout)

       
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)


plot_same_scale(test_1, var=['871e7689cffffff_actual','871e7689cffffff_model'], title = "Demand")
